# Hypergraph product of Expander and 2D Hyperbolic Manifold

Expander graph: Choose a (v,g) cage graph on v vertices of girth g (from https://mathworld.wolfram.com/CageGraph.html) 

Product with 2D  manifold - choose from various families of hyperbolic tesselations formed via the reflection group method of Breuckmann.

Default rainbow code construction is mixed, but users can also compare with the  0-edge contraction mixed or the pin code construction.

In [51]:
from add_parent_dir import *
from common import *
from NHow import *
from CSSLO import *
from flag_codes import *

def E2Mat(E):
    V = {e[0] for e in E}.union({e[1] for e in E})
    V = sorted(V)
    Vdict = {V[i]: i for i in range(len(V))}
    A = ZMatZeros((len(E),len(V)))
    for i in range(len(E)):
        a,b = E[i]
        a,b = Vdict[a],Vdict[b]
        A[i,a],A[i,b] = 1, 1
    return A.T

def cageGraph(v,g):
    if v == 2:
        return repCode(g)
    if g == 2:
        return np.ones((2,v),dtype=int)
    if g == 3:
        return completeGraph(v+1)
    if g == 4:
        return completeBipartiteGraph(v,v)
    if (v,g) == (3,5):
        ## Petersen graph
        E = [[1,2],[1,5],[1,10],[2,9],[2,3],[3,8],[3,4],[4,7],[4,5],[5,6],[6,9],[6,8],[7,9],[7,10],[8,10]]
        return E2Mat(E)
    if (v,g) == (3,6):
        ## Heawood graph
        E = [[1,9],[1,11],[1,10],[2,8],[2,10],[2,12],[3,9],[3,8],[3,13],[4,11],[4,8],[4,14],[5,12],[5,14],[5,9],[6,10],[6,13],[6,14],[7,12],[7,13],[7,11]]
        return E2Mat(E)
    if (v,g) == (3,7):
        ## McGee graph
        LCF,r = [-12,7,-7],8
    if (v,g) == (3,8):
        ## Tutte 8-cage
        LCF,r = [-13,-9,7,-7,9,13],5
    if (v,g) == (4,5):
        ## Robertson graph
        1
    
def completeGraph(v):
    ## complete graph on v+1 vertices has girth 3
    return Mnt(v,2,2).T

def multiGraph(G,u):
    return ZMatVstack([G]*u)

def completeBipartiteGraph(u,v):
    ## bipartite graph on v,v vertices has girth 4
    u1 = np.ones((u,1),dtype=int)
    v1 = np.ones((u,1),dtype=int)
    Iu = ZMatI(u)
    Iv = ZMatI(v)
    U = np.kron(u1,Iv)
    V = np.kron(Iu,v1)
    return ZMatHstack([U,V]).T

def GExplore(A):
    c,n = A.shape
    todo = set(range(c))
    cycles = set()
    PT = dict()
    visitedEdges = set()
    while len(todo) > 0:
        i = min(todo)
        todo.remove(i)
        PT[i] = None
        Rtodo = [i]
        while len(Rtodo) > 0:
            i = Rtodo.pop()
            for c in bin2Set(A[i]):
                for r in bin2Set(A.T[c]):
                    # print(i,r,PT)
                    e = [i,r]
                    e = (min(e),max(e))
                    if i != r and e not in visitedEdges:
                        visitedEdges.add(e)
                        if r not in PT:
                            Rtodo.append(r)
                            todo.remove(r)
                            PT[r] = i
                        else:
                            cycles.add(e)


    return PT,cycles

def Gpath2root(PT,i):
    mypath = [i]
    while i is not None:
        i = PT[i]
        mypath.append(i)
    mypath.reverse()
    return mypath

def Gpath2cycle(p1,p2):
    # print(func_name(),p1,p2)
    i = 0
    nMin = min(len(p1),len(p2))
    while i < nMin and ((p1[i] is None) or (p1[i] == p2[i])):
        i +=1
    p = p1[i:] + p2[i:]
    # print(func_name(),p)
    if i > 0:
        p.append(p1[i-1])
    return p

def Gcycles(A):
    C = []
    PT,cycles = GExplore(A)
    # print('Cycles',cycles)
    for (i,j) in cycles:
        p1,p2 = Gpath2root(PT,i),Gpath2root(PT,j)
        p = Gpath2cycle(p1,p2)
        C.append(p)
    return C

##########################################################
## Reflection Group Codes
##########################################################

## 2D hyperbolic tesselations
# myfile = "RG-4-6.txt"
myfile = "RG-4-5.txt"
# myfile = "RG-5-5.txt"
# myfile = "RG-7-7.txt"

codeList = importRGList(myfile)

print('\n#########################################################')
print(f'## 2D Hyperbolic Tesselations {myfile}')
print(printRGList(codeList,myfile,checkValid=True))
i = 1
complex2D =  codeList[i][1][1:]
SX, SZ = complex2SurfaceCode(codeList[i][1])
SX,LX,SZ,LZ = CSSCode(SX=SX,SZ=SZ)
k,n = LX.shape
b,L = distGenetic(LZ,SZ,tB=1)
dZ = min(np.sum(L,axis=-1))
b,L = distGenetic(LX,SX,tB=1)
dX = min(np.sum(L,axis=-1))
d = min(dZ,dX)
print('\n#########################################################')

A = cageGraph(3,4)
C = Gcycles(A)
cLen = [len(c) for c in C]
print('\n#########################################################')
print(f'2D Code: [[{n},{k},{d}]]')
print(f'Expander dims:{A.shape} nC:{len(C)} girth:{min(cLen)}')

C = complexTCProd(complex2D,[A])

print('\n#########################################################')
constrFun = Complex_to_Code_Mixed
print('\n#########################################################')
print('## Mixed Construction - No Edge Contraction')
res = analyseFlagCode(C,constrFun,calc_dist=False,coloured_logical_paulis=False,calc_LO=False)
print(res)
print('\n#########################################################')
# constrFun = Edge_Contraction_3
# print('\n#########################################################')
# print('## Mixed Construction -  Edge Contraction 0')
# res = analyseFlagCode(C,constrFun,calc_dist=False,coloured_logical_paulis=False,calc_LO=False)
# print(res)
# print('\n#########################################################')
# constrFun = Complex_to_Code_MSG
# print('\n#########################################################')
# print('## MSG Construction - No Edge Contraction')
# res = analyseFlagCode(C,constrFun,calc_dist=False,coloured_logical_paulis=False,calc_LO=False)
# print(res)


#########################################################
## 2D Hyperbolic Tesselations RG-4-5.txt
Codes in File RG-4-5.txt:

i	index	Valid	|C0|	|C1|	|C2|
0	10	True	5	5	1
1	20	True	5	5	2
2	120	True	15	30	12
3	160	True	20	40	16
4	240	True	30	60	24
5	320	True	40	80	32
6	640	True	80	160	64
7	720	True	90	180	72
8	1200	True	150	300	120
9	1320	True	165	330	132
10	1320	True	165	330	132
11	1440	True	180	360	144
12	1920	True	240	480	192
13	2640	True	330	660	264
14	2640	True	330	660	264
15	3840	True	480	960	384
16	5120	True	640	1280	512

#########################################################

#########################################################
2D Code: [[5,0,0]]
Expander dims:(6, 9) nC:4 girth:4

#########################################################

#########################################################
## Mixed Construction - No Edge Contraction

Weights of stabiliser generators and logical Paulis:
SX 8:45,16:45,24:90,30:12,60:18
SZ 4:840,6:240,8:210,10:36
LX 20:1,28:2,108:1,1